In [1]:
import pandas as pd
import json
from openai import OpenAI
import os
from PIL import Image
import matplotlib.pyplot as plt
from glob import glob
import jsonlines
import time


client = OpenAI(
  api_key="your_api_key",
)


NUM_SECONDS_TO_SLEEP = 0.5
def get_eval(content: str, max_tokens: int):
  while True:
    try:
      response = client.chat.completions.create(
        model='gpt-4-1106-preview',
        messages=[{
            'role': 'system',
            'content': '다음 문장을 한국어로 번역해주세요.'
        }, {
            'role': 'user',
            'content': content,
        }],
        temperature=0.2,  # TODO: figure out which temperature is best for evaluation
        max_tokens=max_tokens,
      )
      break
    except openai.RateLimitError:
      pass
    except Exception as e:
      print(e)
    time.sleep(NUM_SECONDS_TO_SLEEP)

  return response.choices[0].message.content


## 텍스트 전처리  
/data/MLP/cschoi/LLaVA/playground/data/coco2014_val_qa_eval/qa90_questions.jsonl  
../playground/data/eval/pope/val2014/COCO_val2014_000000000042.jpg  

In [2]:
import json

dir = '../playground/data/coco2014_val_qa_eval/qa90_questions.jsonl'
img_dir = '../playground/data/eval/pope/val2014/COCO_val2014_'
model_name = 'gpt-4-1106-preview'  # Model name
benchmark_name = 'coco2014_val_qa_eval'  # Benchmark name
language = 'ko'  # Language of the benchmark
output_dir = f'./generated_data/benchmark_data/[{model_name}]_[{benchmark_name}]_[{language}].json'  # Output file path

output_data = []  # List to hold the output data

# Open the JSONL file in read mode
with jsonlines.open(dir, 'r') as reader:
    # Iterate over each line in the file
    for line in reader:
        line['text'] = get_eval(line['text'], 300)
        line['model'] = model_name
        print(line)

        output_data.append(line)  # Add the line to the output data

# Write the output data to a JSON file
with open(output_dir, 'w') as outfile:
    json.dump(output_data, outfile)

{'question_id': 0, 'image': '000000441147.jpg', 'text': '이미지 속 두 개의 여행 가방 색깔은 무엇인가요?', 'category': 'conv', 'model': 'gpt-4-1106-preview'}
{'question_id': 1, 'image': '000000441147.jpg', 'text': '이미지를 종합적이고 상세하게 분석하세요.', 'category': 'detail', 'model': 'gpt-4-1106-preview'}
{'question_id': 2, 'image': '000000441147.jpg', 'text': '이러한 여행 가방이 가치 있게 만들 수 있는 잠재적 요인은 무엇일까요?', 'category': 'complex', 'model': 'gpt-4-1106-preview'}
{'question_id': 3, 'image': '000000353536.jpg', 'text': '이미지 속 테이블 위의 주요 물건들은 무엇인가요?', 'category': 'conv', 'model': 'gpt-4-1106-preview'}
{'question_id': 4, 'image': '000000353536.jpg', 'text': '다음 이미지를 묘사해주세요.', 'category': 'detail', 'model': 'gpt-4-1106-preview'}
{'question_id': 5, 'image': '000000353536.jpg', 'text': '이 테이블 주변에서 최근에 어떤 활동이 있었을까요, 그리고 그 결과는 무엇일까요?', 'category': 'complex', 'model': 'gpt-4-1106-preview'}
{'question_id': 6, 'image': '000000506095.jpg', 'text': '이 이미지의 주요 초점은 무엇인가요?', 'category': 'conv', 'model': 'gpt-4-1106-preview'}
{'question_id': 7,

In [4]:
with open(output_dir, 'w') as outfile:
    json.dump(output_data, outfile, indent=4, ensure_ascii=False)

In [19]:
result =  response.json()

In [22]:
result

{'id': 'chatcmpl-8Mx8BlsBH9mXOjm1jXLnMCYlMutKJ',
 'object': 'chat.completion',
 'created': 1700481207,
 'model': 'gpt-4-1106-vision-preview',
 'usage': {'prompt_tokens': 778,
  'completion_tokens': 119,
  'total_tokens': 897},
 'choices': [{'message': {'role': 'assistant',
    'content': 'This is an image of the Mona Lisa, a famous portrait painting by Leonardo da Vinci. The subject of the painting is a woman, seated against a distant landscape which includes winding paths and a faint blue horizon of mountains. She is notable for her serene and enigmatic expression, leading to much speculation about her mood and thoughts. The Mona Lisa is celebrated for its exquisite detail, use of sfumato (a technique of blending colors and tones), and its subtle modeling of forms. It is one of the most recognized and valuable paintings in the world, displayed at the Louvre Museum in Paris.'},
   'finish_details': {'type': 'stop', 'stop': '<|fim_suffix|>'},
   'index': 0}]}

In [25]:
result['choices'][0]['message']['content']

'This is an image of the Mona Lisa, a famous portrait painting by Leonardo da Vinci. The subject of the painting is a woman, seated against a distant landscape which includes winding paths and a faint blue horizon of mountains. She is notable for her serene and enigmatic expression, leading to much speculation about her mood and thoughts. The Mona Lisa is celebrated for its exquisite detail, use of sfumato (a technique of blending colors and tones), and its subtle modeling of forms. It is one of the most recognized and valuable paintings in the world, displayed at the Louvre Museum in Paris.'